In [3]:
import pandas as pd
import numpy as np
from pydataset import data
import matplotlib.pyplot as plt

# Exercises 1

## 1. Create a function named get_db_url. It should accept a username, hostname, password, and database name and return a url connection string formatted like in the example at the start of this lesson.

In [78]:
from env import host, user, password

In [68]:
def get_db_url(user, host, password, dbname):
    url = f'mysql+pymysql://{user}:{password}@{host}/{dbname}'
    return url

## 2.Use your function to obtain a connection to the employees database.

In [70]:
url = get_db_url(user, host, password, 'employees')

In [42]:
## 3. Once you have successfully run a query:

pd.read_sql('SELECT * FROM salaries LIMIT 5 OFFSET 50', url)

,emp_no,salary,from_date,to_date
0,10005,85076,1993-09-11,1994-09-11
1,10005,86050,1994-09-11,1995-09-11
2,10005,88448,1995-09-11,1996-09-10
3,10005,88063,1996-09-10,1997-09-10
4,10005,89724,1997-09-10,1998-09-10


### a. Intentionally make a typo in the database url. What kind of error message do you see?

In [71]:
url2=get_db_url(user, host, password, 'empoyees')

In [75]:
# pd.read_sql('SELECT * FROM salaries LIMIT 5 OFFSET 50', url2)
# returns OperationalError: (pymysql.err.OperationalError) (1044, "Access denied for user 'xyz'@'%' to database 'empoyees'")


### b. Intentionally make an error in your SQL query. What does the error message look like?

In [76]:
# pd.read_sql('SELECT * FOM salaries LIMIT 5 OFFSET 50', url)
# returns ProgrammingError: (pymysql.err.ProgrammingError) (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax
# to use near 'FOM salaries LIMIT 5 OFFSET 50' at line 1")

## 4. Read the employees and titles tables into two separate DataFrames.

In [77]:
emp_df = pd.read_sql('SELECT * FROM employees LIMIT 5 OFFSET 50', url)
type(emp_df)

pandas.core.frame.DataFrame

In [51]:
emp_df

,emp_no,birth_date,first_name,last_name,gender,hire_date
0,10051,1953-07-28,Hidefumi,Caine,M,1992-10-15
1,10052,1961-02-26,Heping,Nitsch,M,1988-05-21
2,10053,1954-09-13,Sanjiv,Zschoche,F,1986-02-04
3,10054,1957-04-04,Mayumi,Schueller,M,1995-03-13
4,10055,1956-06-06,Georgy,Dredge,M,1992-04-27


In [53]:
titles_df = pd.read_sql('SELECT * FROM titles LIMIT 5 OFFSET 50', url)
titles_df

,emp_no,title,from_date,to_date
0,10036,Senior Staff,1992-04-28,9999-01-01
1,10037,Engineer,1990-12-05,1995-12-05
2,10037,Senior Engineer,1995-12-05,9999-01-01
3,10038,Senior Staff,1996-09-20,9999-01-01
4,10038,Staff,1989-09-20,1996-09-20


## 5. How many rows and columns do you have in each DataFrame? Is that what you expected?

In [54]:
titles_df.shape

(5, 4)

In [55]:
emp_df.shape

(5, 6)

## 6. Display the summary statistics for each DataFrame.

In [56]:
titles_df.describe()

,emp_no
count,5.00000
mean,10037.20000
std,0.83666
min,10036.00000
25%,10037.00000
50%,10037.00000
75%,10038.00000
max,10038.00000


In [57]:
emp_df.describe()

,emp_no
count,5.000000
mean,10053.000000
std,1.581139
min,10051.000000
25%,10052.000000
50%,10053.000000
75%,10054.000000
max,10055.000000


## 7. How many unique titles are in the titles DataFrame?

In [61]:
titles_df.title.unique()

array(['Senior Staff', 'Engineer', 'Senior Engineer', 'Staff'],
      dtype=object)

## 8. What is the oldest date in the to_date column?

In [65]:
titles_df.sort_values(by = ['to_date']).head(1)

,emp_no,title,from_date,to_date
1,10037,Engineer,1990-12-05,1995-12-05


## 9. What is the most recent date in the to_date column?

In [67]:
titles_df.sort_values(by = 'to_date', ascending=False).head(1)

,emp_no,title,from_date,to_date
0,10036,Senior Staff,1992-04-28,9999-01-01


# Exercises 2

## 1. Copy the users and roles DataFrames from the examples above.

In [80]:
# Create the users DataFrame.

users = pd.DataFrame({
    'id': [1, 2, 3, 4, 5, 6],
    'name': ['bob', 'joe', 'sally', 'adam', 'jane', 'mike'],
    'role_id': [1, 2, 3, 3, np.nan, np.nan]
})
users

,id,name,role_id
0,1,bob,1.0
1,2,joe,2.0
2,3,sally,3.0
3,4,adam,3.0
4,5,jane,NaN
5,6,mike,NaN


In [83]:
# Create the roles DataFrame

roles = pd.DataFrame({
    'id': [1, 2, 3, 4],
    'name': ['admin', 'author', 'reviewer', 'commenter']
})
roles

,id,name
0,1,admin
1,2,author
2,3,reviewer
3,4,commenter


## 2. What is the result of using a right join on the DataFrames?

In [85]:
users.merge(roles, left_on='role_id', right_on='id', how='right', indicator=True)
# duplicate column names

,id_x,name_x,role_id,id_y,name_y,_merge
0,1.0,bob,1.0,1,admin,both
1,2.0,joe,2.0,2,author,both
2,3.0,sally,3.0,3,reviewer,both
3,4.0,adam,3.0,3,reviewer,both
4,NaN,NaN,NaN,4,commenter,right_only


## 3. What is the result of using an outer join on the DataFrames?

In [87]:
users.merge(roles, left_on='role_id', right_on='id', how='outer', indicator=True)
# all the rows and columns from both tables

,id_x,name_x,role_id,id_y,name_y,_merge
0,1.0,bob,1.0,1.0,admin,both
1,2.0,joe,2.0,2.0,author,both
2,3.0,sally,3.0,3.0,reviewer,both
3,4.0,adam,3.0,3.0,reviewer,both
4,5.0,jane,NaN,NaN,NaN,left_only
5,6.0,mike,NaN,NaN,NaN,left_only
6,NaN,NaN,NaN,4.0,commenter,right_only


## 4. What happens if you drop the foreign keys from the DataFrames and try to merge them?


In [89]:
users2 = users.copy()
users2

,id,name,role_id
0,1,bob,1.0
1,2,joe,2.0
2,3,sally,3.0
3,4,adam,3.0
4,5,jane,NaN
5,6,mike,NaN


In [98]:
# drop the role_id column
new_users = users2.drop(columns='role_id')
new_users

,id,name
0,1,bob
1,2,joe
2,3,sally
3,4,adam
4,5,jane
5,6,mike


In [100]:
# try to merge the df with foreign key dropped with roles but get a key error
new_users.merge(roles, left_on='role_id', right_on='id', how='outer', indicator=True)

KeyError: 'role_id'

## 5. Load the mpg dataset from PyDataset.

In [103]:
mpg = data('mpg')
mpg

,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact
6,audi,a4,2.8,1999,6,manual(m5),f,18,26,p,compact
7,audi,a4,3.1,2008,6,auto(av),f,18,27,p,compact
8,audi,a4 quattro,1.8,1999,4,manual(m5),4,18,26,p,compact
9,audi,a4 quattro,1.8,1999,4,auto(l5),4,16,25,p,compact
10,audi,a4 quattro,2.0,2008,4,manual(m6),4,20,28,p,compact


## 6. Output and read the documentation for the mpg dataset

In [105]:
data('mpg', show_doc=True)

mpg

PyDataset Documentation (adopted from R Documentation. The displayed examples are in R)

## Fuel economy data from 1999 and 2008 for 38 popular models of car

### Description

This dataset contains a subset of the fuel economy data that the EPA makes
available on http://fueleconomy.gov. It contains only models which had a new
release every year between 1999 and 2008 - this was used as a proxy for the
popularity of the car.

### Usage

    data(mpg)

### Format

A data frame with 234 rows and 11 variables

### Details

  * manufacturer. 

  * model. 

  * displ. engine displacement, in litres 

  * year. 

  * cyl. number of cylinders 

  * trans. type of transmission 

  * drv. f = front-wheel drive, r = rear wheel drive, 4 = 4wd 

  * cty. city miles per gallon 

  * hwy. highway miles per gallon 

  * fl. 

  * class. 




## 7. How many rows and columns are in the dataset?

In [106]:
mpg.shape

(234, 11)

## 8. Check out your column names and perform any cleanup you may want on them.

In [107]:
mpg.columns

Index(['manufacturer', 'model', 'displ', 'year', 'cyl', 'trans', 'drv', 'cty',
       'hwy', 'fl', 'class'],
      dtype='object')

In [124]:
# rename columns
mpg2 = mpg.copy()
mpg2
mpg2.rename(columns={'displ': 'engine displacement', 
                     'cyl': 'cylinder',
                     'trans': 'transmission',
                     'drv': 'drive',
                     'cty': 'city',
                     'hwy': 'highway'})

,manufacturer,model,engine displacement,year,cylinder,transmission,drive,city,highway,fl,class
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact
6,audi,a4,2.8,1999,6,manual(m5),f,18,26,p,compact
7,audi,a4,3.1,2008,6,auto(av),f,18,27,p,compact
8,audi,a4 quattro,1.8,1999,4,manual(m5),4,18,26,p,compact
9,audi,a4 quattro,1.8,1999,4,auto(l5),4,16,25,p,compact
10,audi,a4 quattro,2.0,2008,4,manual(m6),4,20,28,p,compact


## 9. Display the summary statistics for the dataset.

In [108]:
mpg.describe()

,displ,year,cyl,cty,hwy
count,234.000000,234.000000,234.000000,234.000000,234.000000
mean,3.471795,2003.500000,5.888889,16.858974,23.440171
std,1.291959,4.509646,1.611534,4.255946,5.954643
min,1.600000,1999.000000,4.000000,9.000000,12.000000
25%,2.400000,1999.000000,4.000000,14.000000,18.000000
50%,3.300000,2003.500000,6.000000,17.000000,24.000000
75%,4.600000,2008.000000,8.000000,19.000000,27.000000
max,7.000000,2008.000000,8.000000,35.000000,44.000000


## 10. How many different manufacturers are there?

In [114]:
len(mpg.manufacturer.value_counts())

15

## 11. How many different models are there?

In [113]:
len(mpg.model.value_counts())

38

## 12. Create a column named mileage_difference like you did in the DataFrames exercises; this column should contain the difference between highway and city mileage for each car.